In [1]:
import os
import requests
import json
import openai
import pandas as pd
import time
openai.api_key = "a077dc5501894130a1e69037b1e905c9"
openai.api_base = "https://ya2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-08-01-preview'

In [2]:
data = pd.read_csv('data/mc.csv',index_col=0)
data.head()

,qID,sentence,option1,option2,answer,slot_question,slot_answer
0,3FCO4VKOZ4BJQ6IFC0VAIBK4KTWE7U-2,Sarah was a much better surgeon than Maria so ...,Sarah,Maria,2,() was a much better surgeon than Maria so _ a...,Sarah
1,3FCO4VKOZ4BJQ6IFC0VAIBK4KTWE7U-1,Sarah was a much better surgeon than Maria so ...,Sarah,Maria,1,Sarah was a much better surgeon than () so _ a...,Maria
2,3WUVMVA7ODHEES6GZOX75ABL4KQZAX-2,They were worried the wine would ruin the bed ...,blanket,bed,2,They were worried the wine would ruin the bed ...,blanket
3,3B623HUYJ643USRN7YJLDQ8NQH38S9-1,Terry tried to bake the eggplant in the toaste...,eggplant,toaster,1,Terry tried to bake the eggplant in the () ove...,toaster
4,3PN6H8C9R4OWH22DN8WAMK49Q4SADE-1,"At night, Jeffrey always stays up later than H...",Jeffrey,Hunter,1,"At night, Jeffrey always stays up later than (...",Hunter


In [20]:
ans_list = {}

In [16]:
data = data.drop(1059)
data = data.reset_index(drop=True)

In [28]:
for index, row in data[1158:].iterrows():
    init_prompt = "Help me fill in the missing () in option1 or option2.\nHere are some examples format:\n() volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine.\noption1: () \noption2: Dennis \nAnswer:Ian\n"
    prompt = init_prompt + f"{row['slot_question']}\nAnswer:"
    # print(prompt)
    try:
        response = openai.ChatCompletion.create(
            engine="gpt-35-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
    
            ]
        )
    except openai.error.RateLimitError:
            time.sleep(10)
            response = openai.ChatCompletion.create(
            engine="gpt-35-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ]
        )
    print(index,response['choices'][0]['message']['content'])
    ans_list[row['slot_question']] = response['choices'][0]['message']['content']

1158 Ian
1159 Elena is concerned that () might have a Twitter addiction, but () is probably worrying about nothing.
option1: Elena
option2: () 
Answer: someone else
1160 Ian
1161 cups
1162 mixing bowl
1163 The pharmacy offered a product that could cure any disease, made of a new chemical and (herbs), but the () was not FDA approved.
option1: chemical
option2: ()
1164 substance
1165 stale
1166 nails
1167 Someone
1168 to work
ocean
1169 documents
1170 Ian
1171 Ivan
1172 Ian
1173 the police
1174 Someone else
1175 someone
1176 Joseph
Ian
1177 appraiser
1178 clock
1179 hat
1180 test
1181 I cannot determine the missing word in option2 without additional context.
1182 The answer for option1 is "Ian".
1183 Ryan
1184 (option1) Tanya
(option2) Sharon
1185 Ian
1186 energy
1187 The hiking group had less food than (other groups), so they tried to conserve the (remaining supplies) .
1188 someone else
1189 shirt
1190 cousin
1191 syrup
1192 Lawrence volunteered to donate blood to help others, but (Mar

In [29]:
data['pred_answer'] = ans_list.values()
data.to_csv("result/mc_gpt_35.csv")